# Age, Gender, and Player Experience and Data Creation in a Minecraft Server

## Introduction

The question this report deals with is what "kinds" of players, based off their age and played hours have the most sessions on the Minecraft server (i.e. how much data they create). 


In [66]:
library(tidyverse)
library(tidyclust)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard()           masks purrr::discard()
✖ dplyr::filter()             masks stats::filter()
✖ recipes::fixed()            masks stringr::fixed()
✖ parsnip::knit_engine_docs() masks tidyclust::knit_engine_docs()
✖ dplyr::lag()                masks stats::lag()
✖ parsnip::list_md_problems() masks tidyclust::list_md_problems()
✖ yardstick::spec()           masks readr::spec()
✖ recipes::step()             masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/



These are the libraries necessary to perform our wrangling, clustering, and visualization processes.

In [25]:
player_data <- read_csv("data/players.csv")
session_data <- read_csv("data/sessions.csv")

head(player_data)
head(session_data)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail          start_time            end_time original_start_time 
              FALSE               FALSE                TRUE               FALSE 
  original_end_time 
               TRUE

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


We load in, and display the first six rows of each set of data we will be using. There are two sets we use:

Player data:
    - Their experience (Pro, Veteran, Regular, Amateur, or Beginner)
    - Whether they're subscribed to the mailing list (TRUE or FALSE)
    - Their hashed email
    - Hours they have spent playing (>= 0,)
    - Name
    - Gender (male, female, non-binary, two-spirited, agender, other, or prefer not to answer)
    - Age (a whole number, or NA)
    
and session data, which keeps a record of each session, including: 
 - The player's hashed email,
 - Session start time and end time (local time) 
 - Original start and end time (Unix timestamps)

It is notable that in the session data, the end_time and original_end_time columns can contain NA values.

## Methods
First, we will clean our data, and select our necessary columns. For our player data, we will select the columns corresponding to age, played hours, and hashed email (so we can link each player to their sessions). We will also rename the column `Age` to `age` and `hashedEmail` to `hashed_email` for naming convention standardization. We'll also remove any data with an age of NA, as we'll be unable to use this data when we're creating clusters later on.

In [76]:
clean_player_data <- player_data |>
    rename(hashed_email = hashedEmail, age = Age) |>
    select(hashed_email, age, played_hours) |> 
    drop_na(age)

head(clean_player_data)

hashed_email,age,played_hours
<chr>,<dbl>,<dbl>
f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,9,30.3
f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,17,3.8
b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,17,0.0
23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,21,0.7
7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,21,0.1
f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,17,0.0


Next, we'll clean up our session data. We'll rename $hashedEmail$ to $hashed_email$, and only select that column; we're interested in the number of sessions, not the length of sessions.

In [77]:
clean_session_data <- session_data |>
    rename(hashed_email = hashedEmail) |>
    select(hashed_email)
head(clean_session_data)

hashed_email
<chr>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf


Recall that a hashed email is a unique identifier for each player. Since we're interested in the amount of sessions each player created, we'll use $summarise$ with $group_by$ to get the amount of sessions created by each player. 

In [78]:
sessions_by_email <- clean_session_data |>
    group_by(hashed_email) |>
    summarise(session_count = n())
head(sessions_by_email)

hashed_email,session_count
<chr>,<int>
0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc93355ccc95b45423367832,2
060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe8e1cf0eee9a7b67967,1
0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce0290fa0437ce0b97f387,1
0d4d71be33e2bc7266ee4983002bd930f69d304288a8663529c875f40f1750f3,13
0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab9c1ff1a0e7ca200b3a,2
11006065e9412650e99eea4a4aaaf0399bc338006f85e80cc82d18b49f0e2aa4,1


Now that our data is clean, and our session data has hashed_emails linked to their session count, we can combine these two data sets into one, so we can more easily visualize it. We'll do this by matching our `hashed_email`s to each player, and adding the count column. We'll use `left_join` for this; we don't want to lose data from players that have no sessions, but it's okay if we lose data from sessions which we have no player data for (since we can't use this in our analysis). 

We'll also replace any NA counts with zero, because a count of NA means that the player had 0 sessions in our data.

Since our data is combined, we also now can remove `hashed_email`.

In [79]:
combined_session_player_data <- left_join(clean_player_data, sessions_by_email, by = "hashed_email") |>
    mutate(session_count = replace_na(session_count, 0)) |>
    select(-hashed_email)
head(combined_session_player_data)

age,played_hours,session_count
<dbl>,<dbl>,<int>
9,30.3,27
17,3.8,3
17,0.0,1
21,0.7,1
21,0.1,1
17,0.0,0


Our data is now in one usable tibble. We can now start to cluster our data using K-means clustering. First, we create a recipe for our clusters, using all of our predictors. We'll only scale and centre age and session count, since our other two predictors are factors. We'll also create a model specification here (since we're using K-means clustering, we'll use the `k_means` model specification, and since we don't know a good value of K yet, we'll set `num_clusters = tune()`. We will also use the `stats` engine for our clustering.

Note that `ps_*` in the following stands for `player_session_*`, to refer to our combined data.

In [80]:
ps_recipe <- recipe(~ ., data = combined_session_player_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

ps_spec <- k_means(num_clusters = tune()) |>
    set_engine("stats")



Next, we need to find a good value of K for our clusters. We'll ultimately do this by graphing within-cluster sum-of-squared-distances (WSSD) versus K to find where K stops significantly decreasing (the "elbow" of the graph), but we start by picking a range of K values to test (here, 1-10, with a step of 1).

In [81]:
ps_k_vals <- tibble(num_clusters = 1:10)

Now we create a workflow for testing the K values from 1 to 10. We'll add our recipe `ps_recipe`, and our model `ps_spec`, and then use `tune_cluster(resamples = apparent(combined_session_player_data), grid = ps_k_vals)` to tell K-means to run on our combined_session_player_data, and to use the `ps_k_vals` tibble to determine which K values to test. Finally, we'll use `collect_metrics` to actually get the results for each test.

In [85]:
k_val_metrics <- workflow() |>
    add_recipe(ps_recipe) |>
    add_model(ps_spec) |>
    tune_cluster(resamples = apparent(combined_session_player_data), grid = ps_k_vals) |>
    collect_metrics()
k_val_metrics

num_clusters,.metric,.estimator,mean,n,std_err,.config
<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,sse_total,standard,579.00000,1,NA,Preprocessor1_Model01
1,sse_within_total,standard,579.00000,1,NA,Preprocessor1_Model01
2,sse_total,standard,579.00000,1,NA,Preprocessor1_Model02
2,sse_within_total,standard,453.53947,1,NA,Preprocessor1_Model02
3,sse_total,standard,579.00000,1,NA,Preprocessor1_Model03
3,sse_within_total,standard,147.45069,1,NA,Preprocessor1_Model03
4,sse_total,standard,579.00000,1,NA,Preprocessor1_Model04
4,sse_within_total,standard,108.57799,1,NA,Preprocessor1_Model04
5,sse_total,standard,579.00000,1,NA,Preprocessor1_Model05


These metrics aren't very useful to us in their current form, so we'll filter `.metric` to only include the metric `sse_within_total`, which is the only metric we want to look at. We'll also select for only the `num_clusters` and `mean` columns, as those are our relevant data points for graphing. (The `mean` column when `.metric == 'sse_within_total'` is our total WSSD value).

In [86]:
filtered_metrics <- k_val_metrics |>
    filter(.metric == "sse_within_total") |>
    select(num_clusters, mean)
filtered_metrics

num_clusters,mean
<int>,<dbl>
1,579.00000
2,453.53947
3,147.45069
4,108.57799
5,98.48104
6,51.74542
7,71.64186
8,37.80300
9,37.20124


Our data here looks good. Let's graph it to find out where the "elbow" on our plot is.

In [ ]:
k_val_plot <- num_clusters |> 
    filtered_metrics()